In [ ]:
def string_to_search(string, file):
    """Takes in string and file. Finds all lines where that string is. Returns a list of tuples.
    the tuples have the line number and the line where the string appears.
    
    :param string: string to search for
    :param file: location and filename to search
    
    :return: list of tuples, where the tuples have the line numner and line where the string appears
    """
    
    line_number = 0
    list_of_results = []
    with open(file, 'r') as read_obj:
        for line in read_obj:
            line_number += 1
            if string in line:
                list_of_results.append((line_number, line.rstrip()))
                
    return list_of_results

def findData(filename, location):
    """Takes in a txt filename, along with the location on the user's computer. Will find
    the parameters and the spike times. Returns a Data Frame with the spike times and a
    list with the parameters.
    
    :param filename: file name of neuron
    :param loaction: location of file
    
    :return: tuple with the dataframe with spike times and the list of parameters
    """
    
    # Finding kind and neuron name
    neuron_name = ""
    kind = ""
    
    # Kind
    string = filename[::-1]
    while (string[0] != '.'):
        kind += string[0]
        string = string[1:]
    kind = kind[::-1]
    
    # Name
    name = ""
    string = string[1:]
    while (string[0] != '/'):
        name += string[0]
        string = string[1:]
    name = name[::-1]
    name = name[:-2]
    month = ""
    day = ""
    year = ""
    string = filename[10:]
    while (string[0] != '_'):
        month += string[0]
        string = string[1:]
    string = string[1:]
    while (string[0] != '_'):
        day += string[0]
        string = string[1:]
    string = string[1:]
    while (string[0] != '/'):
        year += string[0]
        string = string[1:]
    date = month + '_' + day + '_' + year
    neuron_name = name + " " + date
    
    file = location + filename
    
    # Getting all the lines with "depvar= " to be able to find the parameter list
    list_of_results = string_to_search("depvar= ", file)
    
    paramList = []
    for item in list_of_results:
        string = item[1][8:]
        val = ""
        while string[0] != " ":
            val += string[0]
            string = string[1:]
        paramList.append(int(val))
    
    # Getting all the lines with "nevents=" to find the spike times
    results = string_to_search("nevents=", file)
    list_of_dur = string_to_search("Dur=", file)
    dur = int(list_of_dur[0][1][4:])*10000
    list_of_delay = string_to_search("Delay=", file)
    delay = int(list_of_delay[0][1][6:])*10000
    
    fileOpen = open(file)
    all_lines = fileOpen.readlines()
    fileOpen.close()
    
    spikeList = []
    for item in results:
        number = int(item[1][8:])
        sub_list = []
        for i in range(item[0], item[0] + number):
            string = all_lines[i]
            count = 0
            first_char = string[count]
            val = ''
            while (first_char != '-'):
                val += first_char
                count += 1
                first_char = string[count]
            val = int(val)
            if (val < (dur + delay)) and (val > delay):
                sub_list.append(val/10000)
        spikeList.append(sub_list)
    d = {'spikes': spikeList}       
    spikeDF = pd.DataFrame(d)

    return spikeDF, paramList, neuron_name, kind

def halfWidthFxn(y, x):
    """ This function finds the halfwidth, and is used in returning the
    infoList in the infoFunction. It takes in two lists, the mean count data
    and the list of parameters. It returns a tuple with the HW, LL, UL, and the BV 
    (in that order). """
    
    #Calculating Half Width
    #Subtract off minimum

    y = y - np.min(y)

    #Rectify
    y[y < 0] = 0
    
    #Normalize
    y = y/np.max(y)

    #Interpolate
    xi = np.linspace(np.min(x),np.max(x),1001,endpoint=True)
    yi = np.interp(xi, x, y)

    #Find the maximum value of y
    max_index = np.argmax(yi)

    BV = xi[max_index]

    tol = 0.01
        
    #Cut it in half

    if max_index == 0:

        LL = np.min(xi)

        Ux = xi[max_index+1:]
        Uy = yi[max_index+1:]

        j = np.where(np.abs(Uy -.5) < tol)
    
        while (Ux[j].size <= 0):
            tol += .01
            j = np.where(np.abs(Uy -.5) < tol)
        UL = np.min(Ux[j])
        
    elif max_index == np.size(xi)-1:

        Lx = xi[1:max_index-1]
        Ly = yi[1:max_index-1]

        j = np.where(np.abs(Ly -.5) < tol)
        
        while (Lx[j].size <= 0):
            tol += .01
            j = np.where(np.abs(Ly -.5) < tol)
        LL = np.min(Lx[j])

        UL = np.size(xi)

    else:

        Lx = xi[1:max_index-1]
        Ly = yi[1:max_index-1]

        Ux = xi[max_index+1:]
        Uy = yi[max_index+1:]

        j = np.where(np.abs(Ly -.5) < tol)
       
        while (Lx[j].size <= 0):
            tol += .01
            j = np.where(np.abs(Ly -.5) < tol)
            
        LL = np.max(Lx[j])

        j = np.where(np.abs(Uy -.5) < tol)
        
        tol = 0.01
        while (Ux[j].size <= 0):
            tol += .01
            j = np.where(np.abs(Uy -.5) < tol)
        
        UL = np.min(Ux[j])
    
    #Get half width

    HW=UL-LL
    return HW, LL, UL, BV

def infoFunction(data):
    """ This function takes the data from txt file and returns a list
    of the data values that are extracted. The peak, trough, best value,
    and halfwidth. This function is used in the addingNeuron function. """
    
    # Getting data from tuple
    S = data[0]
    parameters = np.array(data[1])
    spikes = S.to_numpy()
    
    P = parameters.astype(np.float32)
    
    N_trial = np.size(spikes)

    spike_count = np.zeros(N_trial)

    for n in np.arange(N_trial):
        spike_count[n] = np.size(spikes[n][0])

    #Get unique parameter values
    x = np.unique(parameters)

    #Number of unique parameter values
    N = np.size(x)
    mean_count = np.zeros(N)
    sd_count = np.zeros(N)

    for n in np.arange(N):
        index = np.where(parameters == x[n])
    
        mean_count[n] = np.mean(spike_count[index])
    
        sd_count[n] = np.std(spike_count[index],ddof = 1)
        
    HW = round(halfWidthFxn(mean_count, x)[0], 2)
    peak = round(np.max(mean_count), 2)
    trough = round(np.min(mean_count), 2)
    BV = round(halfWidthFxn(mean_count, x)[3], 2)
    
    infoList = [peak, trough, BV, HW, x, mean_count, sd_count]
    
    return infoList

def tuningCurve(file, location):
    
    data = findData(file, location)
    
    # Getting data from tuple
    S = data[0]
    parameters = np.array(data[1])
    spikes = S.to_numpy()
    
    P = parameters.astype(np.float32)
    
    N_trial = np.size(spikes)

    spike_count = np.zeros(N_trial)

    for n in np.arange(N_trial):
        spike_count[n] = np.size(spikes[n][0])

    #Get unique parameter values
    x = np.unique(parameters)

    #Number of unique parameter values
    N = np.size(x)
    mean_count = np.zeros(N)
    sd_count = np.zeros(N)

    for n in np.arange(N):
        index = np.where(parameters == x[n])
    
        mean_count[n] = np.mean(spike_count[index])
    
        sd_count[n] = np.std(spike_count[index],ddof = 1)
     
    #Calculating Half Width
    HW = halfWidthFxn(mean_count, x)[0]
    LL = halfWidthFxn(mean_count, x)[1]
    UL = halfWidthFxn(mean_count, x)[2]
    
    neuron_name = data[2]
    kind = data[3]
        
    units = ""
    if kind == "itd":
        units = ' ($\mu$s)'
    if kind == "iid":
        units = ' (dB)'
    if kind == "bf":
        units = ' (kHz)'
    
    #Plot
    plt.title("Neuron: " + neuron_name, fontsize = 15) 
    plt.errorbar(x,mean_count,sd_count)
    plt.xlabel(kind + units ,fontsize = 15)
    plt.ylabel('Spike count',fontsize = 15)
    plt.axvspan(LL, UL, facecolor='lightblue', alpha=0.5)
    halfWidth = mpatches.Patch(color='lightblue', label='Half Width: ' + str(round(HW, 2)))
    plt.legend(handles=[halfWidth])
    plt.show()
    
def file_table():
    """Returns the filename dataframe. No parameters needed."""
        
    table = pd.DataFrame(np.array([['670.01 12_4_06', 'LLDa/date/12_4_06/12_4_06_data/670.01.0.itd', 'LLDa/date/12_4_06/12_4_06_data/670.01.1.iid', 'LLDa/date/12_4_06/12_4_06_data/670.01.2.bf'],
                          ['670.03 12_4_06', 'LLDa/date/12_4_06/12_4_06_data/670.03.0.itd', 'LLDa/date/12_4_06/12_4_06_data/670.03.1.iid', 'LLDa/date/12_4_06/12_4_06_data/670.03.2.bf'],
                          ['670.04 12_4_06', 'LLDa/date/12_4_06/12_4_06_data/670.04.0.itd', 'LLDa/date/12_4_06/12_4_06_data/670.04.1.iid', 'LLDa/date/12_4_06/12_4_06_data/670.04.2.bf'],
                          ['670.05 12_4_06', 'LLDa/date/12_4_06/12_4_06_data/670.05.0.itd', 'LLDa/date/12_4_06/12_4_06_data/670.05.1.iid', 'LLDa/date/12_4_06/12_4_06_data/670.05.2.bf'],
                          ['670.06 12_4_06', 'LLDa/date/12_4_06/12_4_06_data/670.06.0.itd', None, 'LLDa/date/12_4_06/12_4_06_data/670.06.1.bf'],
                          ['670.01 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.01.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.01.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.01.2.bf'],
                          ['670.02 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.02.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.02.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.02.2.bf'],
                          ['670.03 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.03.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.03.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.03.2.bf'],
                          ['670.04 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.04.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.04.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.04.2.bf'],
                          ['670.05 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.05.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.05.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.05.2.bf'],
                          ['670.06 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.06.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.06.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.06.2.bf'],
                          ['670.08 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.08.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.08.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.08.2.bf'],
                          ['670.09 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.09.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.09.2.iid', 'LLDa/date/11_27_06/11_27_06_data/670.09.3.bf'],
                          ['670.10 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.10.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.10.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.10.2.bf'],
                          ['670.11 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.11.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.11.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.11.2.bf'],
                          ['670.12 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.12.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.12.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.12.2.bf'],
                          ['670.13 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.13.0.itd', None, 'LLDa/date/11_27_06/11_27_06_data/670.13.1.bf'],
                          ['670.14 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.14.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.14.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.14.2.bf'],
                          ['670.15 11_27_06', 'LLDa/date/11_27_06/11_27_06_data/670.15.0.itd', 'LLDa/date/11_27_06/11_27_06_data/670.15.1.iid', 'LLDa/date/11_27_06/11_27_06_data/670.15.2.bf'],
                          ['664.01 10_5_06', 'LLDa/date/10_5_06/10_5_06_data/664.01.0.itd', 'LLDa/date/10_5_06/10_5_06_data/664.01.1.iid', 'LLDa/date/10_5_06/10_5_06_data/664.01.2.bf'],
                          ['664.02 10_5_06', 'LLDa/date/10_5_06/10_5_06_data/664.02.1.itd', 'LLDa/date/10_5_06/10_5_06_data/664.02.0.iid', 'LLDa/date/10_5_06/10_5_06_data/664.02.2.bf'],
                          ['664.04 10_5_06', None, None, 'LLDa/date/10_5_06/10_5_06_data/664.04.1.bf'],
                          ['874.02 10_3_06', 'LLDa/date/10_3_06/10_3_06_data/874.02.0.itd', 'LLDa/date/10_3_06/10_3_06_data/874.02.1.iid', 'LLDa/date/10_3_06/10_3_06_data/874.02.2.bf'],
                          ['874.05 10_3_06', 'LLDa/date/10_3_06/10_3_06_data/874.05.0.itd', 'LLDa/date/10_3_06/10_3_06_data/874.05.1.iid', 'LLDa/date/10_3_06/10_3_06_data/874.05.2.bf'],
                          ['874.07 10_3_06', 'LLDa/date/10_3_06/10_3_06_data/874.07.0.itd', 'LLDa/date/10_3_06/10_3_06_data/874.07.1.iid', 'LLDa/date/10_3_06/10_3_06_data/874.07.2.bf'],
                          ['874.06 9_25_06', 'LLDa/date/9_25_06/9_25_06_data/874.06.0.itd', 'LLDa/date/9_25_06/9_25_06_data/874.06.1.iid', 'LLDa/date/9_25_06/9_25_06_data/874.06.2.bf'],
                          ['874.13 9_25_06', 'LLDa/date/9_25_06/9_25_06_data/874.13.0.itd', 'LLDa/date/9_25_06/9_25_06_data/874.13.1.iid', 'LLDa/date/9_25_06/9_25_06_data/874.13.2.bf'],
                          ['874.03 9_6_06', 'LLDa/date/9_6_06/9_6_06_data/874.03.0.itd', 'LLDa/date/9_6_06/9_6_06_data/874.03.1.iid', 'LLDa/date/9_6_06/9_6_06_data/874.03.2.bf'],
                          ['874.06 9_6_06', 'LLDa/date/9_6_06/9_6_06_data/874.06.0.itd', 'LLDa/date/9_6_06/9_6_06_data/874.06.1.iid', 'LLDa/date/9_6_06/9_6_06_data/874.06.2.bf'],
                          ['874.07 9_6_06', 'LLDa/date/9_6_06/9_6_06_data/874.07.0.itd', 'LLDa/date/9_6_06/9_6_06_data/874.07.1.iid', 'LLDa/date/9_6_06/9_6_06_data/874.07.2.bf'],
                          ['874.09 9_6_06', 'LLDa/date/9_6_06/9_6_06_data/874.09.2.itd', 'LLDa/date/9_6_06/9_6_06_data/874.09.3.iid', 'LLDa/date/9_6_06/9_6_06_data/874.09.5.bf'],
                          ['874.10 9_6_06', 'LLDa/date/9_6_06/9_6_06_data/874.10.0.itd', 'LLDa/date/9_6_06/9_6_06_data/874.10.1.iid', 'LLDa/date/9_6_06/9_6_06_data/874.10.2.bf'],
                          ['874.12 9_6_06', 'LLDa/date/9_6_06/9_6_06_data/874.12.0.itd', 'LLDa/date/9_6_06/9_6_06_data/874.12.1.iid', 'LLDa/date/9_6_06/9_6_06_data/874.12.2.bf'],
                          ['997.02 5_7_07', 'LLDa/date/5_7_07/5_7_07_data/997.02.0.itd', 'LLDa/date/5_7_07/5_7_07_data/997.02.1.iid', 'LLDa/date/5_7_07/5_7_07_data/997.02.2.bf'],
                          ['997.02 3_26_07', 'LLDa/date/3_26_07/3_26_07_data/997.02.0.itd', 'LLDa/date/3_26_07/3_26_07_data/997.02.1.iid', 'LLDa/date/3_26_07/3_26_07_data/997.02.2.bf'],
                          ['997.03 3_26_07', 'LLDa/date/3_26_07/3_26_07_data/997.03.0.itd', 'LLDa/date/3_26_07/3_26_07_data/997.03.1.iid', 'LLDa/date/3_26_07/3_26_07_data/997.03.2.bf'],
                          ['997.04 3_26_07', 'LLDa/date/3_26_07/3_26_07_data/997.04.0.itd', 'LLDa/date/3_26_07/3_26_07_data/997.04.1.iid', 'LLDa/date/3_26_07/3_26_07_data/997.04.2.bf'],
                          ['896.01 1_23_07', 'LLDa/date/1_23_07/1_23_07_data/896.01.0.itd', 'LLDa/date/1_23_07/1_23_07_data/896.01.1.iid', 'LLDa/date/1_23_07/1_23_07_data/896.01.2.bf'],
                          ['896.02 1_23_07', 'LLDa/date/1_23_07/1_23_07_data/896.02.3.itd', 'LLDa/date/1_23_07/1_23_07_data/896.02.1.iid', 'LLDa/date/1_23_07/1_23_07_data/896.02.2.bf'],
                          ['896.03 1_23_07', 'LLDa/date/1_23_07/1_23_07_data/896.03.0.itd', 'LLDa/date/1_23_07/1_23_07_data/896.03.1.iid', 'LLDa/date/1_23_07/1_23_07_data/896.03.2.bf'],
                          ['896.04 1_23_07', 'LLDa/date/1_23_07/1_23_07_data/896.04.0.itd', 'LLDa/date/1_23_07/1_23_07_data/896.04.1.iid', 'LLDa/date/1_23_07/1_23_07_data/896.04.2.bf'],
                          ['896.05 1_23_07', 'LLDa/date/1_23_07/1_23_07_data/896.05.3.itd', 'LLDa/date/1_23_07/1_23_07_data/896.05.1.iid', 'LLDa/date/1_23_07/1_23_07_data/896.05.2.bf'],
                          ['896.07 1_23_07', 'LLDa/date/1_23_07/1_23_07_data/896.07.2.itd', 'LLDa/date/1_23_07/1_23_07_data/896.07.1.iid', 'LLDa/date/1_23_07/1_23_07_data/896.07.4.bf'],
                          ['995.01 1_8_07', 'LLDa/date/1_8_07/1_8_07_data/995.01.0.itd', 'LLDa/date/1_8_07/1_8_07_data/995.01.1.iid', 'LLDa/date/1_8_07/1_8_07_data/995.01.2.bf']]),
                   columns=['name', 'itd', 'iid', 'bf'])
    return table
    


In [ ]:
def printDataframe(location):
    """"""
    files = file_table()
    
    names = []
    itdPeak_list = []
    itdTrough_list = []
    itdBV_list = []
    itdHW_list = []
    iidPeak_list = []
    iidTrough_list = []
    iidBV_list = []
    iidHW_list = []
    bfPeak_list = []
    bfTrough_list = []
    bfBV_list = []
    bfHW_list = []
    
    for index, row in files.iterrows():
        # Setting names
        names.append(row['name'])
        
        # Getting file names for each itd, iid, bf
        itdFile = row['itd']
        iidFile = row['iid']
        bfFile = row['bf']
        
        # Adding itd data to lists
        if itdFile == None:
            itdPeak_list.append(None)
            itdTrough_list.append(None)
            itdBV_list.append(None)
            itdHW_list.append(None)
        else:
            data = findData(itdFile, location)
            info = infoFunction(data)
        
            itdPeak_list.append(info[0])
            itdTrough_list.append(info[1])
            itdBV_list.append(info[2])
            itdHW_list.append(info[3])
        
        # Adding iid data to lists
        if iidFile == None:
            iidPeak_list.append(None)
            iidTrough_list.append(None)
            iidBV_list.append(None)
            iidHW_list.append(None)
        else:
            data = findData(iidFile, location)
            info = infoFunction(data)
        
            iidPeak_list.append(info[0])
            iidTrough_list.append(info[1])
            iidBV_list.append(info[2])
            iidHW_list.append(info[3])
        
        # Adding bf data to lists
        if bfFile == None:
            bfPeak_list.append(None)
            bfTrough_list.append(None)
            bfBV_list.append(None)
            bfHW_list.append(None)
        else:
            data = findData(bfFile, location)
            info = infoFunction(data)
        
            bfPeak_list.append(info[0])
            bfTrough_list.append(info[1])
            bfBV_list.append(info[2])
            bfHW_list.append(info[3])
    
    final = pd.DataFrame({"Name": names,"itd Peak": itdPeak_list, "itd Trough": itdTrough_list, "itd Best Vaule": itdBV_list, "itd Halfwidth": itdHW_list, "iid Peak": iidPeak_list, "iid Trough": iidTrough_list, "iid Best Vaule": iidBV_list, "iid Halfwidth": iidHW_list, "bf Peak": bfPeak_list, "bf Trough": bfTrough_list, "bf Best Vaule": bfBV_list, "bf Halfwidth": bfHW_list})
    
    return final

def filename_search(name):
    """takes in the name of the nueron and searches through the dataframe to 
    return the filenmaes for itd, iid, and bf. Returns a list with these strings in that order. If
    the file does not exist the list will hold the value None. If the name doesn't exist, will return
    an empty list."""
    
    table = file_table()
    
    if name in table.values:
        index = table[table.name == name].index[0]
        return [table.iloc[index, 1], table.iloc[index, 2], table.iloc[index, 3]]
    else:
        return []
    
def tuningCurveName(name, kind, location):
     
    alt_return = ""
    
    if kind == 'itd':
        index = 0
    elif kind == 'iid':
        index = 1
    elif kind == 'bf':
        index = 2
    else:
        alt_return += "Invalid kind"
    
    file = filename_search(name)
    if file == []:
        if len(alt_return) > 0:
            alt_return += " and invalid name"
        else:
            alt_return += "Invalid name"
            
    if len(alt_return) > 0:
        return alt_return
    else:
        file = file[index]
        return tuningCurve(file, location)
    
class Data:
    def __init__(self, name, kind, location):
        if kind == 'itd':
            index = 0
        elif kind == 'iid':
            index = 1
        elif kind == 'bf':
            index = 2
        file = filename_search(name)[index]
        data = findData(file, location)
        info = infoFunction(data)
        self.peak = info[0]
        self.trough = info[1]
        self.BV = info[2]
        self.HW = info[3]
        self.parameters = info[4]
        self.mean_count = info[5]
        self.sd = info[6]

def getData(name, kind, location):
    return Data(name, kind, location)
      